<a href="https://colab.research.google.com/github/rohihilel/auto-correct-whatsapp-message/blob/main/titanic_roee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [698]:
import pandas as pd
import numpy as np
import time
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
path = "/content/drive/MyDrive/datasets/titanic.csv"
df = pd.read_csv(path)

1.יש לוודא שהמידע תקין, אין בו כפילויות, חוסרים או ערכים תקולים:


In [699]:
df['PassengerId'].value_counts()

1309    1
449     1
431     1
432     1
433     1
       ..
874     1
875     1
876     1
877     1
1       1
Name: PassengerId, Length: 1309, dtype: int64

בדקתי על כל עמודה ורק בשמות מצאתי כפילויות - אין כפילויות

In [700]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     1309 non-null   int64  
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 122.8+ KB


ישנם 1309 שורות, לכן ניתן לראות שבעמודה של הגיל יש חוסר מפני שיש 1046 במקום 1309, גם בעוד כמה עמודות יש חוסרים אז  לכן נחשב את הממוצע של איפה שיש חסרים בעמודות של מספרים כגון: AGE , FARE

In [701]:
avrage_age=df['Age'].mean()
avrage_fare=df['Fare'].mean()

נמלא איפה שחסר

In [702]:
df['Age'].fillna(value=avrage_age, inplace=True)
df['Fare'].fillna(value=avrage_fare, inplace=True)

embark היא עמודה ללא מספר אלא אותיות לכן נשתמש בפונקציה שלוקחת את הערך הקודם

In [703]:
df['Embarked'].fillna( method ='ffill', limit = 1, inplace = True)

embark is about where the person got to the ship C = Cherbourg; Q = Queenstown; S = Southampton
זה לא מספר או כן ולא ולכן נבדוק את הערכים ונשנה בהתאם

In [704]:
df['Embarked'].value_counts(normalize=True)

S    0.698243
C    0.207028
Q    0.094729
Name: Embarked, dtype: float64

נשנה

In [705]:
S   = 0.698243
C    =0.207028
Q   = 0.094729
df["Embarked"].replace({"S": 0.698243, "C": 0.207028, "Q":0.094729}, inplace=True)

נהפוך את הערכים של מין לבוליאן נכון ולא נכון

In [706]:
df['isMale'] = df['Sex']=='male'
df.drop('Sex', axis=1, inplace=True)

remove the cabin section cause there is a lot missing

In [707]:
df.drop('Cabin', axis=1, inplace=True)

נבדוק

In [708]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     1309 non-null   int64  
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Age          1309 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1309 non-null   float64
 9   Embarked     1309 non-null   float64
 10  isMale       1309 non-null   bool   
dtypes: bool(1), float64(3), int64(5), object(2)
memory usage: 103.7+ KB


כמו שניתן לראות לא חסר שום דבר עכשיו

In [709]:
df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Embarked,isMale
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,0.698243,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,0.207028,False
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,0.698243,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,0.698243,False
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,0.698243,True


עכשיו צריך להפריד את הערכים ל :


*   XY
ולהפריד את הדברים החשובים. לפי דעתי הדברים החשובים אצלנו זה הדמי כניסה, גיל, מין, מקורבים ומקום עלייה, כל השאר הם דברים פרטיים ששונים לכל אחד כמו שם ושם כרטיס



In [710]:
X = df.iloc[:,[False,False,False,False,True,True,True,False,True,True,True]].values
Y = df.iloc[:, 1].values
print("start: ",X.shape, Y.shape)

start:  (1309, 6) (1309,)


נחליף את מה שכתוב בזכר נקבה במקום נכון לא נכון ל 0 1

In [711]:
labelencoder_X = LabelEncoder()
X[:, 5] = labelencoder_X.fit_transform(X[:, 5])
print(Y)

[0 1 1 ... 0 0 0]


נשנה את סוגי המספרים לפלואט, ככה שהמחשב  יוכל לקרוא את זה וניתן לעמודות שם בטבלה החדשה

In [712]:
Y1 = Y.reshape((len(Y),1))

np_arr = np.concatenate((Y1,X),axis=1)
df1 = pd.DataFrame(np_arr)

df1.columns = ['Survived','Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'isMale']
df1 = df1.astype('float64')
print(df1['isMale'])

print(df1.corr()['Survived'][:].sort_values(ascending=False))


0       1.0
1       0.0
2       0.0
3       0.0
4       1.0
       ... 
1304    1.0
1305    0.0
1306    1.0
1307    1.0
1308    1.0
Name: isMale, Length: 1309, dtype: float64
Survived    1.000000
Fare        0.233568
Parch       0.108919
SibSp       0.002370
Age        -0.048483
Embarked   -0.133001
isMale     -0.688371
Name: Survived, dtype: float64


as we can see, SidSp and Age dont infect

בגלל שלמחשב לא קל לקרוא מספרים גבוהים נשתמש בסיפריה סקייל של 

sklearn 

ונעשה למשתנים התאמה עשרונית חוץ ממשתנה המין שהוא לא תלוי אלא הוא כן ולא

In [713]:
print(df1)
scaler = MinMaxScaler()
df1[['Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'isMale']] = scaler.fit_transform(df1[['Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'isMale']])

Y=df1['Survived']
xc = df1[['Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'isMale']]
print(xc)


      Survived        Age  SibSp  Parch      Fare  Embarked  isMale
0          0.0  22.000000    1.0    0.0    7.2500  0.698243     1.0
1          1.0  38.000000    1.0    0.0   71.2833  0.207028     0.0
2          1.0  26.000000    0.0    0.0    7.9250  0.698243     0.0
3          1.0  35.000000    1.0    0.0   53.1000  0.698243     0.0
4          0.0  35.000000    0.0    0.0    8.0500  0.698243     1.0
...        ...        ...    ...    ...       ...       ...     ...
1304       0.0  29.881138    0.0    0.0    8.0500  0.698243     1.0
1305       1.0  39.000000    0.0    0.0  108.9000  0.207028     0.0
1306       0.0  38.500000    0.0    0.0    7.2500  0.698243     1.0
1307       0.0  29.881138    0.0    0.0    8.0500  0.698243     1.0
1308       0.0  29.881138    1.0    1.0   22.3583  0.207028     1.0

[1309 rows x 7 columns]
           Age  SibSp     Parch      Fare  Embarked  isMale
0     0.273456  0.125  0.000000  0.014151  1.000000     1.0
1     0.473882  0.125  0.000000  0.1391

נחלק לאימון ומבחן

In [714]:
X_train, X_test, Y_train, Y_test = train_test_split(xc, Y, test_size = 0.2)

In [715]:
print("train: ", X_train.shape, Y_train.shape)
print("test:  " , X_test.shape, Y_test.shape)


train:  (1047, 6) (1047,)
test:   (262, 6) (262,)


for every k from 3 to 10 with jump netween 2 we are training the mashine, checking the score of the test and another check is: metricks accurcy score

In [716]:
for k in range(3,10,2):
    knn3 = KNeighborsClassifier(n_neighbors = k)  
    knn3.fit(X_train,Y_train)     
    print("k = {} score test: {}".format(k,knn1.score(X_test, Y_test)))  
    print("metrics:")
    y_pred = knn3.predict(X_test)
    print(metrics.accuracy_score(Y_test, y_pred))
    print("\n"+"*"*15+"\n")
    time.sleep(1) 

k = 3 score test: 0.9083969465648855
metrics:
0.816793893129771

***************

k = 5 score test: 0.9083969465648855
metrics:
0.8244274809160306

***************

k = 7 score test: 0.9083969465648855
metrics:
0.8435114503816794

***************

k = 9 score test: 0.9083969465648855
metrics:
0.8549618320610687

***************

